In [68]:
%pylab inline
import numpy.linalg as LA
import codecs, json
from time import time
import plotly.express as px
import re
import string
import requests
import collections
import math
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random', 'time']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$




---



---


# Q3: Natural Language Processing (****) (60pt)


One of applications of Markov chain in natural language processing is language model. In particular, we will work on uni-gram and bi-gram.

The following data set provides you with the training data files (a subset of the One Billion Word Language Modeling Bench-
mark). Each line in each file contains a whitespace-tokenized sentence.

- `1b benchmark.train.tokens`: data for training your language models.
- `1b benchmark.dev.tokens`: data for debugging and choosing the best hyperparameters.
- `1b benchmark.test.tokens`: data for evaluating your language models.


I have precessed these dataset for the purposed of this problem. The total number of unique words is 80661. In bi-gram model, the size of the transition matrix is $80661\times 80661$. It is impossible the store these matrices directly and we have to take advantage of the sparsity of the transition matrix.

You should use the development data to choose the best values for the hyperparameters $\lambda$. Hyperparameter
optimization is an active area of research; for this homework, you can simply try a few combinations to find
reasonable values.

In [ ]:
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.train.tokens?raw=true -O 1b_benchmark.train.tokens
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.dev.tokens?raw=true -O 1b_benchmark.dev.tokens
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.test.tokens?raw=true -O 1b_benchmark.test.tokens


--2023-11-10 06:06:45--  https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.train.tokens?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8422714 (8.0M) [text/plain]
Saving to: ‘1b_benchmark.train.tokens’

1b_benchmark.train. 100%[===================>]   8.03M  --.-KB/s    in 0.1s    

2023-11-10 06:06:47 (65.8 MB/s) - ‘1b_benchmark.train.tokens’ saved [8422714/8422714]

--2023-11-10 06:06:47--  https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.dev.tokens?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:4

In [ ]:
START_TOKEN = "<s>"
STOP_TOKEN = "</s>"

class FileParser:
    def __init__(self, train_file="1b_benchmark.train.tokens", test_file="1b_benchmark.test.tokens", dev_file="1b_benchmark.dev.tokens"):
        self.TRAIN_FILE = train_file
        self.TEST_FILE  = test_file
        self.DEV_FILE = dev_file

    def get_train_file_tokens(self):
        return self._tokenize(self._get_sentences(self.TRAIN_FILE))

    def get_dev_file_tokens(self):
        return self._tokenize(self._get_sentences(self.DEV_FILE))

    def get_test_file_tokens(self):
        return self._tokenize(self._get_sentences(self.TEST_FILE))

    def get_train_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.TRAIN_FILE), flatten=False)

    def get_dev_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.DEV_FILE), flatten=False)

    def get_test_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.TEST_FILE), flatten=False)

    def _flatten(self, l):
        return [word for sublist in l for word in sublist]

    def _tokenize(self, sentence_list, flatten=True):
        tokenized_sentences = [re.split("\s+", sentence.strip()) for sentence in sentence_list]

        if flatten:
            return self._flatten(tokenized_sentences)

        return tokenized_sentences

    def _get_sentences(self, file_path):

        l = []
        with open(file_path, "r") as f:
            l = f.readlines()

        # Add the start and stop tokens to each sentence in the file
        sentence_list = []
        for sentence in l:
            sentence_list.append(START_TOKEN + " " + sentence + " " + STOP_TOKEN)

        return sentence_list

In [ ]:
fp = FileParser()
train_tokens = fp.get_train_file_sentence_tokens()
dev_tokens = fp.get_dev_file_sentence_tokens()
test_tokens = fp.get_test_file_sentence_tokens()


In [ ]:
# Building the dict and the map from the words to integers for the purpose of training.
word2idx={}
idx2word={}
idx=0
for line in train_tokens:
    for word in line:
        if word not in word2idx:
            word2idx[word]=idx
            idx2word[idx]=word
            idx+=1

# Building the dict and the map from the words to integers for dev.
word2idx_dev={}
idx2word_dev={}
idx=0
for line in dev_tokens:
    for word in line:
        if word not in word2idx_dev:
            word2idx_dev[word]=idx
            idx2word_dev[idx]=word
            idx+=1

# Transform the words in each sentence to integers
train_data=[]
for line in train_tokens:
    train_data.append([word2idx[word] for word in line])

dev_data=[]
for line in dev_tokens:
    dev_data.append([word2idx_dev[word] for word in line])





---



---


**Your task:** Complete the code in the ```LanguageModel``` class as follows.

1. (15pt) Implement **Laplace Smoothing** with the smoothing factor =1 in the function ```get_unigram_probability``` and ```get_bigram_probability```. In the bigram model, try to implement it as efficient as possible. **Hint:** For bigrams $(t_1,t_2)$ which do not occur in the sample, what is $p(t_2|t_1)$? For fixed $t_1$, are these probability the same?

2. (15pt) Complete functions ```get_unigram_sentence_log_probability``` and ```get_bigram_sentence_log_probability``` to calculate the log probability of the given sentence. You need to consider the situation that when Laplace Smoothing is true and the situation that when Laplace Smoothing is false.


3. (10pt) To make your language model work better, you will implement linear interpolation smoothing between
unigram, bigram.
\begin{align}
p'(t_2|t_1) = \lambda_1 p(t_2) + \lambda_2 p(t_2|t_1)
\end{align}
where $p'$ represents the smoothed probability, the hyperparameters $\lambda_1, \lambda_2$ are weights on the unigram,
bigram language models, respectively. So $\lambda_1+\lambda_2= 1$.
Complete functions ```get_linear_interpolation_probability``` and ```get_linear_interpolation_sentence_log_probability``` with $\mm\lambda = (\lambda_1, \lambda_2)$ stored in ```linear_interpolation_factors```.

4. (5pt) Testing the smoothed probability with several sentences in the development dataset. You should test when laplace smoothing is True and when laplace smoothing is False. You might find some words in testing dataset are not appeared in training set and you can set a default probability for this situation.  

3. (15pt) Report the **perplexity scores** of the linear interpolation of language model for your training,
and development sets. Report no
more than 5 different sets of $\lambda$. Briefly discuss the experimental results. Putting it all together, report perplexity on the test set, using the hyperparameters that you chose from
the development set. Specify those hyperparameters.

In [72]:
class LanguageModel(object):

    def __init__(self, laplace_smoothing=False, laplace_smoothing_factor=1):
        self.unigram_freqs = {}
        self.unigram_corpus_length = 0
        self.num_unique_unigrams = 0

        self.bigram_freqs = {}
        self.bigram_corpus_length = 0
        self.num_unique_bigrams = 0

        self.laplace_smoothing = laplace_smoothing
        self.laplace_smoothing_factor = laplace_smoothing_factor


    def fit_unigram(self, tokens):
        for clist in tokens:
          for cindex in range(len(clist)):
            t = clist[cindex]
            self.unigram_freqs[(t)] = self.unigram_freqs.get((t),0) + 1

        self.unigram_corpus_length = sum(list(self.unigram_freqs.values()))
        self.num_unique_unigrams = len(list(self.unigram_freqs.keys()))


    def fit_bigram(self, tokens):

      for clist in tokens:
        for cindex in range(0, len(clist) - 1):
          t1, t2 =  clist[cindex], clist[cindex+1]
          self.bigram_freqs[(t1, t2)] = self.bigram_freqs.get((t1, t2), 0) + 1

      self.bigram_corpus_length = sum(list(self.bigram_freqs.values()))
      self.num_unique_bigrams = len(list(self.bigram_freqs.keys()))


    def get_unigram_probability(self, unigram):

      if self.laplace_smoothing:
        prob_numerator   = 1 + self.unigram_freqs.get(unigram, 0)
        prob_denominator = self.num_unique_unigrams + self.unigram_corpus_length
        prob = float(prob_numerator) / float(prob_denominator)
        return prob

      else:
        prob_numerator   = self.unigram_freqs.get(unigram, 0)
        prob_denominator = self.unigram_corpus_length
        prob = float(prob_numerator) / float(prob_denominator)
        return prob


    def get_bigram_probability(self, bigram):
      t1, t2 = bigram

      if self.laplace_smoothing:
        prob_numerator   = 1 + self.bigram_freqs.get((t1, t2), 0)
        prob_denominator = self.num_unique_unigrams + self.unigram_freqs.get(t1, 0)
        prob = float(prob_numerator) / float(prob_denominator)
        return prob
      else:
        prob_numerator   = self.bigram_freqs.get((t1, t2), 0)
        prob_denominator = self.unigram_freqs.get(t1, 0)
        if prob_denominator == 0:
            print(f"Error: 'get_bigram_probability()' has a denominator of 0 for {bigram}")
            return float('inf')

        prob = float(prob_numerator) / float(prob_denominator)
        return prob


    def get_linear_interpolation_probability(self, bigram, lif):
      t1,t2 = bigram
      lif1,lif2 = lif
      value = (lif1*self.get_unigram_probability(t2))+(lif2*self.get_bigram_probability((t1,t2)))
      return value


    def get_unigram_sentence_log_probability(self, sentence):

      sum=0
      for x in sentence:
        sum+=math.log(self.get_unigram_probability(x), 2)
      return sum


    def get_bigram_sentence_log_probability(self, sentence):

      sum=0
      for x1 in range(len(sentence)-1):
        sum+=math.log(self.get_bigram_probability((sentence[x1],sentence[x1+1])), 2)
      return sum


    def get_linear_interpolation_sentence_log_probability(self, sentence, lif):

      sum=0
      for x1 in range(len(sentence)-1):
        sum+=math.log(self.get_linear_interpolation_probability((sentence[x1],sentence[x1+1]), lif), 2)
      return sum


    def get_unigram_perplexity(self, sentences):

      scores=[]
      for x in sentences:
        score=pow(2, (((-1)/len(x))*self.get_unigram_sentence_log_probability(x[1])))
        scores.append([x[0], score])
      return scores


    def get_bigram_perplexity(self, sentences):

      scores=[]
      for x in sentences:
        score=pow(2, (((-1)/len(x))*self.get_bigram_sentence_log_probability(x[1])))
        scores.append([x[0], score])
      return scores


    def get_linear_interpolation_perplexity(self, sentences, lif):

      scores=[]
      for x in sentences:
        score=pow(2, (((-1)/len(x))*self.get_linear_interpolation_sentence_log_probability(x[1], lif)))
        scores.append([x[0], score])
      return scores



In [73]:
import random

lmf=LanguageModel()
lmt=LanguageModel(laplace_smoothing=True)

lmf.fit_unigram(dev_data)
lmf.fit_bigram(dev_data)

lmt.fit_unigram(dev_data)
lmt.fit_bigram(dev_data)

i=0
tot_diff=0
num_sentences = 500

while i<num_sentences:
  temp_index=random.randint(0,len(dev_data)-1)
  temp_sentence = dev_data[temp_index]
  difference=lmf.get_linear_interpolation_sentence_log_probability(temp_sentence, (.3,.7))- lmt.get_linear_interpolation_sentence_log_probability(temp_sentence, (.3,.7))
  tot_diff+=difference
  i+=1

print("The average difference between no laplace smoothing and laplace smoothing in the log probability is", tot_diff/num_sentences)
# upon taking a difference between the two, we see that with laplace smoothing being true, we get a LOWER log probability in general for all sentences.

The average difference between no laplace smoothing and laplace smoothing in the log probability is 125.06179490158868


In [74]:
lm=LanguageModel(laplace_smoothing=True)

lm.fit_unigram(train_data)
lm.fit_bigram(train_data)

i=0
num_sentences = 5
sentence_list = []

while i<num_sentences:
  temp_index=random.randint(0,len(dev_data)-1)
  sentence_list.append([temp_index, dev_data[temp_index]])
  i+=1

print(lm.get_unigram_perplexity(sentence_list))
print(lm.get_bigram_perplexity(sentence_list))
print(lm.get_linear_interpolation_perplexity(sentence_list, (.49, .51)))
print(lm.get_linear_interpolation_perplexity(sentence_list, (.3, .7)))
print(lm.get_linear_interpolation_perplexity(sentence_list, (.7, .3)))
print(lm.get_linear_interpolation_perplexity(sentence_list, (.1, .9)))
print(lm.get_linear_interpolation_perplexity(sentence_list, (.9, .1)))

[[11787, 2.4990909668821693e+33], [635, 7.079568752532694e+149], [10410, 5.4869755697143696e+53], [1371, 2.7793730533368023e+84], [5202, 5.599665377937189e+23]]
[[11787, 3.80824022207682e+36], [635, 7.919038681593329e+169], [10410, 1.3567359281568313e+63], [1371, 5.882952711336944e+86], [5202, 4.928135215677749e+24]]
[[11787, 1.9722815196886856e+32], [635, 5.823930127774991e+148], [10410, 1.3066027843997777e+53], [1371, 1.726917879165502e+81], [5202, 5.727970432496406e+22]]
[[11787, 4.605252129296569e+32], [635, 2.3409743135225913e+151], [10410, 1.4116193720545303e+54], [1371, 3.5451181192633427e+81], [5202, 8.020462502452456e+22]]
[[11787, 1.5537763163244355e+32], [635, 2.8775498844731327e+147], [10410, 3.8483179088682046e+52], [1371, 3.0935743955657546e+81], [5202, 5.654926129689887e+22]]
[[11787, 5.05099289725323e+33], [635, 2.0277018635593446e+157], [10410, 4.321806127979578e+56], [1371, 6.989470518225716e+82], [5202, 2.339954155823195e+23]]
[[11787, 2.3848911575016435e+32], [635, 

# Q4: (Bonus) Beam Search Revisited (*****) (15pt)
Implement Beam Search Algorithm for Q3. Note you need to use various data structure and graphs to reduce the complexity of implementation.


In [ ]:
# Your code starts here.
